# <div dir="rtl">الوصول إلى بيانات MERRA-2 في السحابة</div>

<div dir="rtl">
*جزء من:* [**البيانات المناخية المفتوحة**](https://github.com/OpenClimateScience/M1-Open-Climate-Data) | **الدرس السابق** | **الدرس التالي**
</div>

<div dir="rtl">
**المحتويات:**

- [استخدام `earthaccess`](#Using-earthaccess)
- [الحصول على سلسلة زمنية لدرجات الحرارة](#Getting-a-temperature-time-series)
</div>

In [ ]:
import earthaccess
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

<div dir="rtl">
## استخدام `earthaccess`
</div>

<div dir="rtl">
في الدرس السابق، قمنا بتنزيل ملف netCDF4 يدويًا من بحث Earthdata التابع لناسا. الآن، لنرى كيف يمكننا بدلاً من ذلك استخدام كود Python لتنزيل البيانات من السحابة.

يمكن استخدام مكتبة `earthaccess` للبحث وتنزيل البيانات من بحث Earthdata دون فتح متصفح الويب. بالنسبة لبعض التطبيقات، يمكن أن يمكننا هذا من كتابة كود Python يكون أكثر شفافية وقابلية للتكرار، لأنه يسمح لشخص آخر بالحصول بسهولة على نفس البيانات الخام التي نبدأ بها.

نستخدم `earthaccess.search_data()` للحصول على واحد أو أكثر من **البيانات الجزئية** التي تطابق استعلام البحث. البيانات الجزئية هي مجموعة بيانات فردية مرتبطة بمهمة أو منتج معين من ناسا.

**هل تتذكر أين يتم العثور على الاسم القصير لمنتجات Earthdata؟** في صفحة المعلومات (i)، هناك موقعان:

![](./assets/M1_screenshot_Earthdata_Search_MERRA2_info.png)

**نستخدم الاسم القصير `short_name` لتحديد مجموعة البيانات الجزئية التي نرغب في البحث عنها.**

في هذا المثال، نجد أن هناك 32 بيانات جزئية بين 1 مايو و 1 يونيو، شاملاً.
</div>

In [ ]:
results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2023-05", "2023-06"))
results[0]

<div dir="rtl">
لم يتم تنزيل أي بيانات بعد؛ لدينا فقط مرجع للبيانات المخزنة في السحابة.
</div>

In [ ]:
type(results[0])

<div dir="rtl">
لتنزيل البيانات الجزئية، يمكننا استخدام `earthaccess.open()`.

#### &#x1F6A9; <span style="color:red">انتبه جيدًا</span>

**شيء مهم يجب ملاحظته حول `earthaccess.open()` هو أنه يتطلب *تسلسلاً* من البيانات الجزئية لفتحها.** لذلك، حتى إذا كنا نرغب في فتح جزء واحد فقط، يجب تقديم هذا الجزء كجزء من قائمة أو مجموعة في Python.
</div>

In [ ]:
# ملاحظة: open() يتطلب تسلسلاً من مراجع الملفات
files = earthaccess.open(results[0:2])

In [ ]:
type(files[0])

<div dir="rtl">
تم تنزيل البيانات الجزئية في ذاكرة الكمبيوتر الخاص بنا. لفتح أحد الملفات التي تم تنزيلها في Python، نستخدم الآن `xarray`. هناك تأخير طفيف عند استخدام `open_dataset()` على البيانات الجزئية التي تم تنزيلها باستخدام `earthaccess.open()` لأن `xarray` يجب أن يحلل الملف ويفهم نظام الإحداثيات والأبعاد.
</div>

In [ ]:
ds2 = xr.open_dataset(files[0])
ds2

<div dir="rtl">
مجموعة بيانات `xarray.Dataset` الناتجة جاهزة للتخطيط!
</div>

In [ ]:
ds2['T2MMIN'].plot()

## <div dir="rtl">الحصول على سلسلة زمنية لدرجات الحرارة</div>


<div dir="rtl">
لقد عمل هذا بشكل رائع لملف واحد، ولكن ماذا لو أردنا إنشاء سلسلة زمنية من بيانات المناخ؟ نعلم أن مجموعات بيانات `xarray` يمكن أن تحتوي على بُعد زمني، مما يجعلها قادرة على تمثيل أكثر من حالة واحدة في الوقت. كيف يمكننا إنشاء مثل هذه المجموعة من البيانات؟

في هذا المثال التالي، نستخدم حلقة `for` للتكرار عبر بيانات MERRA-2 الجزئية، وفتح كل منها ثم اختيار قيمة `T2MMIN` (درجة الحرارة الدنيا) عند موقع محدد. نضيف هذه القيمة إلى قائمة، جنبًا إلى جنب مع تاريخ (`"time"`) البيانات الجزئية، لبناء مجموعة بيانات للسلسلة الزمنية.
</div>

In [ ]:
# ملاحظة: قد يستغرق هذا المثال نصف دقيقة مع اتصال إنترنت جيد.
results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2023-05", "2023-06"))

time_list = []
data_list = []
file_list = earthaccess.open(results)
for filename in file_list:
    ds = xr.open_dataset(filename)
    data_list.append(ds['T2MMIN'].sel(lat = 36.5, lon = 3.125).values)
    time_list.append(ds['T2MMIN']['time'].values)

<div dir="rtl">
لدينا الآن قائمة Python تحتوي على قيم درجات الحرارة الدنيا وقائمة أخرى من التواريخ. أدناه، نرسم هذه البيانات بعد استخدام `numpy` لتحويل بيانات درجات الحرارة من كلفن (K) إلى درجات مئوية (C).
</div>

In [ ]:
import numpy as np

# تحويل من كلفن إلى مئوية
data = np.array(data_list).ravel() - 273.15
time = np.array(time_list).ravel()

pyplot.figure(figsize = (12, 4))
pyplot.plot(time, data)
pyplot.ylabel('درجة الحرارة الدنيا (°C)')